In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, average_precision_score

In [2]:
df = pd.read_csv('./data/S05-hw-dataset.csv')
df.head()

,client_id,age,income,years_employed,credit_score,debt_to_income,num_credit_cards,num_late_payments,has_mortgage,has_car_loan,savings_balance,checking_balance,region_risk_score,phone_calls_to_support_last_3m,active_loans,customer_tenure_years,default
0,1,25,94074,22,839,0.547339,1,7,0,0,26057,5229,0.080052,19,1,8,0
1,2,58,51884,26,565,0.290882,1,1,0,1,16221,11595,0.428311,15,0,7,0
2,3,53,48656,39,561,0.522340,1,13,0,0,55448,-2947,0.770883,15,4,5,0
3,4,42,81492,30,582,0.709123,2,10,1,1,35188,17727,0.357619,0,2,7,1
4,5,42,94713,8,642,0.793392,3,3,0,0,0,-404,0.414260,17,1,10,1


In [3]:
df.describe()

,client_id,age,income,years_employed,credit_score,debt_to_income,num_credit_cards,num_late_payments,has_mortgage,has_car_loan,savings_balance,checking_balance,region_risk_score,phone_calls_to_support_last_3m,active_loans,customer_tenure_years,default
count,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,1500.500000,45.059667,69658.992000,19.577667,649.285333,0.284065,3.494667,6.869333,0.495000,0.501333,20607.256667,5559.684333,0.400175,9.524667,1.976333,6.968667,0.410333
std,866.169729,14.192883,24742.235182,11.381497,69.955852,0.161112,2.289917,4.291278,0.500058,0.500082,14035.209739,6306.032612,0.204529,5.779030,1.408700,4.349942,0.491976
min,1.000000,21.000000,15000.000000,0.000000,402.000000,0.006147,0.000000,0.000000,0.000000,0.000000,0.000000,-3000.000000,0.001148,0.000000,0.000000,0.000000,0.000000
25%,750.750000,33.000000,52641.750000,10.000000,604.000000,0.157796,1.000000,3.000000,0.000000,0.000000,9612.250000,341.500000,0.239208,5.000000,1.000000,3.000000,0.000000
50%,1500.500000,45.000000,69784.500000,20.000000,647.000000,0.261726,3.000000,7.000000,0.000000,1.000000,20021.000000,5114.500000,0.381992,10.000000,2.000000,7.000000,0.000000
75%,2250.250000,57.000000,85874.250000,29.000000,697.000000,0.388886,6.000000,10.000000,1.000000,1.000000,30101.250000,9906.250000,0.549213,15.000000,3.000000,11.000000,1.000000
max,3000.000000,69.000000,156351.000000,39.000000,850.000000,0.878343,7.000000,14.000000,1.000000,1.000000,75237.000000,29335.000000,0.961733,19.000000,4.000000,14.000000,1.000000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   client_id                       3000 non-null   int64  
 1   age                             3000 non-null   int64  
 2   income                          3000 non-null   int64  
 3   years_employed                  3000 non-null   int64  
 4   credit_score                    3000 non-null   int64  
 5   debt_to_income                  3000 non-null   float64
 6   num_credit_cards                3000 non-null   int64  
 7   num_late_payments               3000 non-null   int64  
 8   has_mortgage                    3000 non-null   int64  
 9   has_car_loan                    3000 non-null   int64  
 10  savings_balance                 3000 non-null   int64  
 11  checking_balance                3000 non-null   int64  
 12  region_risk_score               30

In [5]:
print(df.value_counts(normalize=True)) # Checking class balance
print(df.isnull().sum())  # Checking for missing values
print(df.duplicated().sum())  # Checking for duplicate rows

client_id  age  income  years_employed  credit_score  debt_to_income  num_credit_cards  num_late_payments  has_mortgage  has_car_loan  savings_balance  checking_balance  region_risk_score  phone_calls_to_support_last_3m  active_loans  customer_tenure_years  default
1          25   94074   22              839           0.547339        1                 7                  0             0             26057             5229             0.080052           19                              1             8                      0          0.000333
2004       43   35470   26              538           0.307111        0                 11                 0             1             7070              4144             0.377450           0                               4             5                      1          0.000333
1995       57   106544  25              602           0.362523        4                 0                  1             0             27045             8240             0.326452

In [6]:
print(df['default'].value_counts(normalize=True))  # Checking class balance for target variable

default
0    0.589667
1    0.410333
Name: proportion, dtype: float64


датасет в норме, пропущенных значений нет, категориальных признаков нет, дубликатов не имеется

числовые значения находится в нормальных диапозонах, аномалий нет

всего 3000 объектов в выборке

баланс классов нормальный

In [7]:
df.head()

,client_id,age,income,years_employed,credit_score,debt_to_income,num_credit_cards,num_late_payments,has_mortgage,has_car_loan,savings_balance,checking_balance,region_risk_score,phone_calls_to_support_last_3m,active_loans,customer_tenure_years,default
0,1,25,94074,22,839,0.547339,1,7,0,0,26057,5229,0.080052,19,1,8,0
1,2,58,51884,26,565,0.290882,1,1,0,1,16221,11595,0.428311,15,0,7,0
2,3,53,48656,39,561,0.522340,1,13,0,0,55448,-2947,0.770883,15,4,5,0
3,4,42,81492,30,582,0.709123,2,10,1,1,35188,17727,0.357619,0,2,7,1
4,5,42,94713,8,642,0.793392,3,3,0,0,0,-404,0.414260,17,1,10,1


In [8]:
# обучение
X, y = df.drop(columns=['client_id', 'default']), df['default']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [9]:
# бэйзлайн
dummy_clf = DummyClassifier(strategy='uniform')
dummy_clf.fit(X_train, y_train)
dummy_preds = dummy_clf.predict(X_test)
acc_dummy = accuracy_score(y_test, dummy_preds)
f1_dummy = f1_score(y_test, dummy_preds)
precision_dummy = precision_score(y_test, dummy_preds, zero_division=0)
recall_dummy = recall_score(y_test, dummy_preds)
print(f'Dummy Classifier Accuracy: {acc_dummy:.4f}')
print(f'Dummy Classifier F1 Score: {f1_dummy:.4f}')
print(f'Dummy Classifier Precision: {precision_dummy:.4f}')
print(f'Dummy Classifier Recall: {recall_dummy:.4f}')

Dummy Classifier Accuracy: 0.4867
Dummy Classifier F1 Score: 0.4144
Dummy Classifier Precision: 0.3893
Dummy Classifier Recall: 0.4431


из-за того, что баланс классов примерно равный, доля правильных ответов бэйза = 0.5 (что соответствует случайному угадыванию класса)


попробуем теперь обучить линейную модель

In [20]:
# лог рег
log_reg_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=1000, tol=1e-3))
])

# сгд с логлоссом
sgd_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('sgd', SGDClassifier(loss='log_loss', max_iter=1000, tol=1e-3))
])

In [21]:
# подбор для лог рег
search_log_reg = GridSearchCV(
    estimator=log_reg_pipeline,
    param_grid = {
    'logreg__C': np.logspace(-3, 3, 7),
    'logreg__penalty': ['l1'],
    'logreg__solver': ['liblinear', 'saga'],  # только эти солверы для L1
    'logreg__max_iter': [1000, 2000],
    'logreg__class_weight': [None, 'balanced']
},
    scoring='f1',
    cv=5,
    n_jobs=-1
)

# подбор для сгд
search_sgd = GridSearchCV(
    estimator=sgd_pipeline,
    param_grid={
        'sgd__alpha': [0.0001, 0.001, 0.01],
        'sgd__learning_rate': ['optimal', 'invscaling', 'constant'],
        'sgd__eta0': [0.01, 0.1, 1.0]
    },
    scoring='f1',
    cv=5,
    n_jobs=-1
)

In [22]:
search_log_reg.fit(X_train, y_train)
search_sgd.fit(X_train, y_train)


,estimator,Pipeline(step...'log_loss'))])
,param_grid,"{'sgd__alpha': [0.0001, 0.001, ...], 'sgd__eta0': [0.01, 0.1, ...], 'sgd__learning_rate': ['optimal', 'invscaling', ...]}"
,scoring,'f1'
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,copy,True


In [23]:
best_sgd = search_sgd.best_estimator_
best_log_reg = search_log_reg.best_estimator_
best_sgd_preds = best_sgd.predict(X_test)
best_log_reg_preds = best_log_reg.predict(X_test)


In [24]:
sgd_acc = accuracy_score(y_test, best_sgd_preds)
sgd_f1 = f1_score(y_test, best_sgd_preds)
sgd_roc_auc = roc_auc_score(y_test, best_sgd.predict_proba(X_test)[:, 1])
print(f'SGD Classifier Accuracy: {sgd_acc:.4f}')
print(f'SGD Classifier F1 Score: {sgd_f1:.4f}')
print(f'SGD Classifier ROC AUC: {sgd_roc_auc:.4f}')
print(f'SGD precision: {precision_score(y_test, best_sgd_preds, zero_division=0):.4f}')
print(f'SGD recall: {recall_score(y_test, best_sgd_preds):.4f}')

SGD Classifier Accuracy: 0.7967
SGD Classifier F1 Score: 0.7382
SGD Classifier ROC AUC: 0.8730
SGD precision: 0.7818
SGD recall: 0.6992


In [25]:
log_acc = accuracy_score(y_test, best_log_reg_preds)
log_f1 = f1_score(y_test, best_log_reg_preds)
log_roc_auc = roc_auc_score(y_test, best_log_reg.predict_proba(X_test)[:, 1])
print(f'Logistic Regression Accuracy: {log_acc:.4f}')
print(f'Logistic Regression F1 Score: {log_f1:.4f}')
print(f'Logistic Regression ROC AUC: {log_roc_auc:.4f}')
print(f'Logistic Regression precision: {precision_score(y_test, best_log_reg_preds, zero_division=0):.4f}')
print(f'Logistic Regression recall: {recall_score(y_test, best_log_reg_preds):.4f}')

Logistic Regression Accuracy: 0.7967
Logistic Regression F1 Score: 0.7540
Logistic Regression ROC AUC: 0.8753
Logistic Regression precision: 0.7480
Logistic Regression recall: 0.7602


как видим из результатов метрик, линейная модель справляется уже сильно лучше, чем просто случайное угадывание (хотя всё равно плохо)

это видно из более высоких значений метрик f1-score и прочих метрик

для данной задачи очевидно лучше справляется логистическая регрессия, чем бейзлайн модель, потому что она нашла какие-то закономерности в данных, а не просто угадывает ответ или предсказывает его константой

In [ ]:
# ROC и PR кривые для лучшей модели (Logistic Regression) и Dummy классификатора
from sklearn.metrics import roc_curve, precision_recall_curve, average_precision_score

# Получаем вероятности для моделей
dummy_probs = dummy_clf.predict_proba(X_test)[:, 1]
log_reg_probs = best_log_reg.predict_proba(X_test)[:, 1]
sgd_probs = best_sgd.predict_proba(X_test)[:, 1]

# ROC кривые
fpr_dummy, tpr_dummy, _ = roc_curve(y_test, dummy_probs)
fpr_log_reg, tpr_log_reg, _ = roc_curve(y_test, log_reg_probs)
fpr_sgd, tpr_sgd, _ = roc_curve(y_test, sgd_probs)

# PR кривые
precision_dummy, recall_dummy, _ = precision_recall_curve(y_test, dummy_probs)
precision_log_reg, recall_log_reg, _ = precision_recall_curve(y_test, log_reg_probs)
precision_sgd, recall_sgd, _ = precision_recall_curve(y_test, sgd_probs)

# Построение графиков
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# ROC кривая
axes[0].plot(fpr_dummy, tpr_dummy, label=f'Dummy (AUC = {roc_auc_score(y_test, dummy_probs):.3f})', linestyle='--')
axes[0].plot(fpr_log_reg, tpr_log_reg, label=f'Logistic Regression (AUC = {roc_auc_score(y_test, log_reg_probs):.3f})')
axes[0].plot(fpr_sgd, tpr_sgd, label=f'SGD (AUC = {roc_auc_score(y_test, sgd_probs):.3f})')
axes[0].plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random Classifier')
axes[0].set_xlabel('False Positive Rate')
axes[0].set_ylabel('True Positive Rate')
axes[0].set_title('ROC Curve')
axes[0].legend()
axes[0].grid(True)

# PR кривая
axes[1].plot(recall_dummy, precision_dummy, label=f'Dummy (AP = {average_precision_score(y_test, dummy_probs):.3f})', linestyle='--')
axes[1].plot(recall_log_reg, precision_log_reg, label=f'Logistic Regression (AP = {average_precision_score(y_test, log_reg_probs):.3f})')
axes[1].plot(recall_sgd, precision_sgd, label=f'SGD (AP = {average_precision_score(y_test, sgd_probs):.3f})')
axes[1].set_xlabel('Recall')
axes[1].set_ylabel('Precision')
axes[1].set_title('Precision-Recall Curve')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig('figures/roc_pr_curves.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Сравнение метрик Dummy и LogisticRegression
print('Сравнение метрик:')
print(f'Dummy Accuracy: {acc_dummy:.4f}, ROC-AUC: {roc_auc_score(y_test, dummy_probs):.4f}')
print(f'Logistic Regression Accuracy: {log_acc:.4f}, ROC-AUC: {log_roc_auc:.4f}')
print(f'SGD Accuracy: {sgd_acc:.4f}, ROC-AUC: {sgd_roc_auc:.4f}')


# Финальный отчет

В ходе выполнения домашнего задания была решена задача бинарной классификации - предсказание дефолта клиента банка. Были построены и протестированы две линейные модели: логистическая регрессия и SGD-классификатор.

Модель логистической регрессии показала Accuracy: 0.7967 и ROC-AUC: 0.8753, что значительно превышает результат бейзлайн-модели Dummy, у которой Accuracy составил 0.4867 и ROC-AUC: 0.5000 примерно.

SGD-классификатор показал Accuracy: 0.7967 и ROC-AUC: 0.8730, что также значительно лучше бейзлайн-модели.

Таким образом, обе линейные модели успешно справились с задачей и показали существенное улучшение по сравнению с бейзлайн-моделью, что говорит о наличии закономерностей в данных, которые модели смогли выявить.


In [ ]:
# ROC и PR кривые для лучшей модели (Logistic Regression) и Dummy классификатора
from sklearn.metrics import roc_curve, precision_recall_curve, average_precision_score

# Получаем вероятности для моделей
dummy_probs = dummy_clf.predict_proba(X_test)[:, 1]
log_reg_probs = best_log_reg.predict_proba(X_test)[:, 1]
sgd_probs = best_sgd.predict_proba(X_test)[:, 1]

# ROC кривые
fpr_dummy, tpr_dummy, _ = roc_curve(y_test, dummy_probs)
fpr_log_reg, tpr_log_reg, _ = roc_curve(y_test, log_reg_probs)
fpr_sgd, tpr_sgd, _ = roc_curve(y_test, sgd_probs)

# PR кривые
precision_dummy, recall_dummy, _ = precision_recall_curve(y_test, dummy_probs)
precision_log_reg, recall_log_reg, _ = precision_recall_curve(y_test, log_reg_probs)
precision_sgd, recall_sgd, _ = precision_recall_curve(y_test, sgd_probs)

# Построение графиков
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# ROC кривая
axes[0].plot(fpr_dummy, tpr_dummy, label=f'Dummy (AUC = {roc_auc_score(y_test, dummy_probs):.3f})', linestyle='--')
axes[0].plot(fpr_log_reg, tpr_log_reg, label=f'Logistic Regression (AUC = {roc_auc_score(y_test, log_reg_probs):.3f})')
axes[0].plot(fpr_sgd, tpr_sgd, label=f'SGD (AUC = {roc_auc_score(y_test, sgd_probs):.3f})')
axes[0].plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random Classifier')
axes[0].set_xlabel('False Positive Rate')
axes[0].set_ylabel('True Positive Rate')
axes[0].set_title('ROC Curve')
axes[0].legend()
axes[0].grid(True)

# PR кривая
axes[1].plot(recall_dummy, precision_dummy, label=f'Dummy (AP = {average_precision_score(y_test, dummy_probs):.3f})', linestyle='--')
axes[1].plot(recall_log_reg, precision_log_reg, label=f'Logistic Regression (AP = {average_precision_score(y_test, log_reg_probs):.3f})')
axes[1].plot(recall_sgd, precision_sgd, label=f'SGD (AP = {average_precision_score(y_test, sgd_probs):.3f})')
axes[1].set_xlabel('Recall')
axes[1].set_ylabel('Precision')
axes[1].set_title('Precision-Recall Curve')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig('figures/roc_pr_curves.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Сравнение метрик Dummy и LogisticRegression
print('Сравнение метрик:')
print(f'Dummy Accuracy: {acc_dummy:.4f}, ROC-AUC: {roc_auc_score(y_test, dummy_probs):.4f}')
print(f'Logistic Regression Accuracy: {log_acc:.4f}, ROC-AUC: {log_roc_auc:.4f}')
print(f'SGD Accuracy: {sgd_acc:.4f}, ROC-AUC: {sgd_roc_auc:.4f}')


# Финальный отчет

В ходе выполнения домашнего задания была решена задача бинарной классификации - предсказание дефолта клиента банка. Были построены и протестированы две линейные модели: логистическая регрессия и SGD-классификатор.

Модель логистической регрессии показала Accuracy: 0.7967 и ROC-AUC: 0.8753, что значительно превышает результат бейзлайн-модели Dummy, у которой Accuracy составил 0.4867 и ROC-AUC: 0.5000 примерно.

SGD-классификатор показал Accuracy: 0.7967 и ROC-AUC: 0.8730, что также значительно лучше бейзлайн-модели.

Таким образом, обе линейные модели успешно справились с задачей и показали существенное улучшение по сравнению с бейзлайн-моделью, что говорит о наличии закономерностей в данных, которые модели смогли выявить.
